In [ ]:
# KCB AI & Data Science

# Predicting Diabetes Risk Using Patient Health Indicators and AI Models

# Submitted By
# Name: GILBERT WALIUBA
# Email: gilbert@haofinder.com
# Phone: +254 715 560 734

#  Institution
# ADNIAN LABS

# Date of Submission
# 15th SEPT 2025

# Dataset Source: Pima Indians Diabetes Database

"""
Project structure
- Data loading
- Data inspection
- Preprocessing (missing values, scaling)
- Exploratory Data Analysis (plots)
- Feature engineering
- Model training (Logistic Regression, Decision Tree, Random Forest, SVM, MLP)
- Evaluation (confusion matrix, classification report, ROC-AUC)
- Hyperparameter tuning (GridSearchCV)
- Cross-validation
- Feature importance and model saving
- Generate a short markdown report file

Notes for Google Colab:
- Run each cell sequentially. If running as a script, plotting windows will appear inline in Jupyter.
- This script uses common libraries: pandas, numpy, matplotlib, seaborn, scikit-learn, joblib.
"""

# ---------------------------
# 0. Imports and settings
# ---------------------------
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (confusion_matrix, classification_report, roc_auc_score,
                             RocCurveDisplay, PrecisionRecallDisplay, accuracy_score)
import joblib

# Display settings
pd.set_option('display.max_columns', None)
sns.set(style='whitegrid')

# Create output directory
OUT_DIR = Path('diabetes_capstone_outputs')
OUT_DIR.mkdir(exist_ok=True)

# ---------------------------
# 1. Load dataset
# ---------------------------
# Using the common Pima Indians dataset CSV hosted at a stable URL
URL = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ["Pregnancies","Glucose","BloodPressure","SkinThickness",
           "Insulin","BMI","DiabetesPedigreeFunction","Age","Outcome"]

print('Loading dataset...')
df = pd.read_csv(URL, names=columns)
print('Loaded:', df.shape)

# Quick peek
print(df.head())

# Save a copy
df.to_csv(OUT_DIR / 'raw_pima_diabetes.csv', index=False)

# ---------------------------
# 2. Initial inspection
# ---------------------------
print('\nData info:')
print(df.info())
print('\nStatistics:')
print(df.describe().T)

# Check target balance
print('\nOutcome distribution:')
print(df['Outcome'].value_counts())

# ---------------------------
# 3. Data cleaning & preprocessing
# ---------------------------
# Replace biologically-impossible zeroes with NaN for specific columns
cols_with_zero = ["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]
df[cols_with_zero] = df[cols_with_zero].replace(0, np.nan)

print('\nMissing value counts after replacing zeroes:')
print(df.isna().sum())

# Impute missing values with median (robust for skewed data)
imputer = SimpleImputer(strategy='median')
df[cols_with_zero] = imputer.fit_transform(df[cols_with_zero])

print('\nMissing value counts after imputation:')
print(df.isna().sum())

# Sanity: check distributions after imputation
print('\nPost-imputation descriptive stats:')
print(df[cols_with_zero].describe().T)

# Feature matrix and target
X = df.drop('Outcome', axis=1).copy()
y = df['Outcome'].copy()

# Scale numeric features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Save preprocessed data
pd.concat([X_scaled, y.reset_index(drop=True)], axis=1).to_csv(OUT_DIR / 'preprocessed_pima_diabetes.csv', index=False)

# ---------------------------
# 4. Exploratory Data Analysis (EDA)
# ---------------------------
# Target distribution plot
plt.figure(figsize=(6,4))
sns.countplot(x=y)
plt.title('Outcome Distribution (0: No Diabetes, 1: Diabetes)')
plt.savefig(OUT_DIR / 'outcome_distribution.png')
plt.close()

# Histograms for features
X.hist(bins=20, figsize=(12,10))
plt.suptitle('Feature Distributions (raw scale)')
plt.savefig(OUT_DIR / 'feature_histograms.png')
plt.close()

# Correlation heatmap (on raw X)
plt.figure(figsize=(10,8))
sns.heatmap(pd.concat([X, y], axis=1).corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.savefig(OUT_DIR / 'correlation_heatmap.png')
plt.close()

# Boxplots to show potential outliers
plt.figure(figsize=(12,8))
X.boxplot()
plt.xticks(rotation=45)
plt.title('Boxplots of features')
plt.savefig(OUT_DIR / 'boxplots_features.png')
plt.close()

# Pairplot on a sample (to keep it fast)
sample = pd.concat([X.sample(250, random_state=42), y.sample(250, random_state=42).reset_index(drop=True)], axis=1)
# pairplot can be slow; we save only if seaborn supports it quickly
try:
    sns.pairplot(sample, hue='Outcome', corner=True)
    plt.savefig(OUT_DIR / 'pairplot_sample.png')
    plt.close()
except Exception:
    pass

# ---------------------------
# 5. Feature engineering
# ---------------------------
# Example features: Age group, BMI x Age interaction
X_fe = X.copy()
# Age group bins
age_bins = [20, 30, 40, 50, 60, 100]
age_labels = ['20-29','30-39','40-49','50-59','60+']
X_fe['AgeGroup'] = pd.cut(X_fe['Age'], bins=age_bins, labels=age_labels, right=False)

# Interaction term
X_fe['BMI_Age'] = X_fe['BMI'] * X_fe['Age']

# One-hot encode AgeGroup
X_fe = pd.get_dummies(X_fe, columns=['AgeGroup'], drop_first=True)

# Scale again after engineering
X_fe_scaled = pd.DataFrame(scaler.fit_transform(X_fe), columns=X_fe.columns)

# Save engineered features
pd.concat([X_fe_scaled, y.reset_index(drop=True)], axis=1).to_csv(OUT_DIR / 'engineered_pima_diabetes.csv', index=False)

# ---------------------------
# 6. Train-test split
# ---------------------------
X_train, X_test, y_train, y_test = train_test_split(X_fe_scaled, y, test_size=0.2, stratify=y, random_state=42)
print('\nTrain/Test sizes:', X_train.shape, X_test.shape)

# ---------------------------
# 7. Model definitions and helper functions
# ---------------------------
models = {
    'LogisticRegression': LogisticRegression(max_iter=2000),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'RandomForest': RandomForestClassifier(random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'MLP': MLPClassifier(max_iter=1000, random_state=42)
}


def evaluate_model(name, model, Xt, yt, Xv, yv, plot_roc=True):
    model.fit(Xt, yt)
    preds = model.predict(Xv)
    probs = None
    try:
        probs = model.predict_proba(Xv)[:,1]
    except Exception:
        try:
            probs = model.decision_function(Xv)
        except Exception:
            probs = None

    print(f"\nModel: {name}")
    print('Accuracy:', accuracy_score(yv, preds))
    print(classification_report(yv, preds))
    if probs is not None:
        print('ROC-AUC:', roc_auc_score(yv, probs))
        # Plot ROC
        plt.figure(figsize=(6,5))
        RocCurveDisplay.from_estimator(model, Xv, yv)
        plt.title(f'ROC Curve - {name}')
        plt.savefig(OUT_DIR / f'roc_{name}.png')
        plt.close()

    # Confusion matrix
    cm = confusion_matrix(yv, preds)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.savefig(OUT_DIR / f'cm_{name}.png')
    plt.close()

    return model

# ---------------------------
# 8. Train baseline models
# ---------------------------
trained_models = {}
for name, mdl in models.items():
    trained_models[name] = evaluate_model(name, mdl, X_train, y_train, X_test, y_test)

# ---------------------------
# 9. Cross-validation scores
# ---------------------------
print('\nCross-validation (5-fold) scores:')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_results = {}
for name, mdl in models.items():
    scores = cross_val_score(mdl, X_fe_scaled, y, cv=skf, scoring='roc_auc')
    cv_results[name] = scores
    print(f"{name}: mean ROC-AUC = {scores.mean():.4f} | std = {scores.std():.4f}")

# ---------------------------
# 10. Hyperparameter tuning for top models
# ---------------------------
# We will tune RandomForest and LogisticRegression as primary candidates
param_grid_rf = {
    'n_estimators': [100, 200],
    'max_depth': [None, 6, 10],
    'min_samples_split': [2, 5]
}

param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['lbfgs']
}

print('\nTuning RandomForest...')
rf = RandomForestClassifier(random_state=42)
grid_rf = GridSearchCV(rf, param_grid_rf, cv=skf, scoring='roc_auc', n_jobs=-1)
grid_rf.fit(X_train, y_train)
print('Best RF params:', grid_rf.best_params_)
print('Best RF ROC-AUC:', grid_rf.best_score_)

print('\nTuning LogisticRegression...')
lr = LogisticRegression(max_iter=2000)
grid_lr = GridSearchCV(lr, param_grid_lr, cv=skf, scoring='roc_auc', n_jobs=-1)
grid_lr.fit(X_train, y_train)
print('Best LR params:', grid_lr.best_params_)
print('Best LR ROC-AUC:', grid_lr.best_score_)

# Evaluate tuned models on test set
best_rf = grid_rf.best_estimator_
best_lr = grid_lr.best_estimator_

best_rf = evaluate_model('RandomForest_Tuned', best_rf, X_train, y_train, X_test, y_test)
best_lr = evaluate_model('LogisticRegression_Tuned', best_lr, X_train, y_train, X_test, y_test)

# ---------------------------
# 11. Feature importance (Random Forest)
# ---------------------------
if hasattr(best_rf, 'feature_importances_'):
    fi = pd.Series(best_rf.feature_importances_, index=X_fe_scaled.columns).sort_values(ascending=False)
    print('\nTop features by importance:')
    print(fi.head(15))
    plt.figure(figsize=(8,6))
    fi.head(15).plot(kind='bar')
    plt.title('Feature Importances - Random Forest')
    plt.tight_layout()
    plt.savefig(OUT_DIR / 'feature_importances_rf.png')
    plt.close()

# ---------------------------
# 12. Save best model and scalers
# ---------------------------
model_path = OUT_DIR / 'best_model_random_forest.joblib'
joblib.dump(best_rf, model_path)
scaler_path = OUT_DIR / 'scaler.joblib'
joblib.dump(scaler, scaler_path)
print(f'Best model saved to: {model_path}')

# ---------------------------
# 13. Simple prediction function
# ---------------------------

def predict_patient(data_dict, model=best_rf, scaler=scaler, feature_columns=X_fe.columns):
    # data_dict: dict with keys matching original feature names (Pregnancies, Glucose, ... Age)
    df_input = pd.DataFrame([data_dict])
    # Impute zeros if present (simple approach)
    for c in cols_with_zero:
        if c in df_input.columns and df_input.loc[0, c] == 0:
            df_input.loc[0, c] = np.nan
    df_input[cols_with_zero] = imputer.transform(df_input[cols_with_zero])
    # Feature engineering: AgeGroup & BMI_Age
    df_input['BMI_Age'] = df_input['BMI'] * df_input['Age']
    df_input['AgeGroup'] = pd.cut(df_input['Age'], bins=age_bins, labels=age_labels, right=False)
    df_input = pd.get_dummies(df_input, columns=['AgeGroup'], drop_first=True)
    # Ensure all expected columns exist
    for col in feature_columns:
        if col not in df_input.columns:
            df_input[col] = 0
    df_input = df_input[feature_columns]
    # Scale
    Xs = scaler.transform(df_input)
    pred_prob = model.predict_proba(Xs)[:,1][0]
    pred = int(pred_prob >= 0.5)
    return {'prediction': pred, 'probability': float(pred_prob)}

# Example usage of predict_patient
sample_patient = {
    'Pregnancies': 2,
    'Glucose': 120,
    'BloodPressure': 70,
    'SkinThickness': 20,
    'Insulin': 79,
    'BMI': 27.5,
    'DiabetesPedigreeFunction': 0.5,
    'Age': 29
}
print('\nSample prediction:', predict_patient(sample_patient))

# ---------------------------
# 14. Generate short markdown report
# ---------------------------
report_md = f"""
# Diabetes Prediction Capstone - Short Report

**Dataset:** Pima Indians Diabetes Database (768 records)

**Preprocessing:** Zero values in biologically-impossible fields replaced with median. Standard scaling applied. Feature engineering added BMI_Age and AgeGroup bins.

**Models trained:** Logistic Regression, Decision Tree, Random Forest, Support Vector Machine, Multi-layer Perceptron.

**Best tuned model:** Random Forest (saved in {model_path})

**Key performance (test set):**
"""

# append test results summary

def summarize_metrics(model, X_test, y_test):
    preds = model.predict(X_test)
    try:
        probs = model.predict_proba(X_test)[:,1]
        auc = roc_auc_score(y_test, probs)
    except Exception:
        auc = None
    acc = accuracy_score(y_test, preds)
    rpt = classification_report(y_test, preds, output_dict=True)
    return acc, auc, rpt

acc_rf, auc_rf, rpt_rf = summarize_metrics(best_rf, X_test, y_test)

report_md += f"- Random Forest accuracy: {acc_rf:.3f}\n"
if auc_rf is not None:
    report_md += f"- Random Forest ROC-AUC: {auc_rf:.3f}\n"

report_md += '\n## Top features (Random Forest)\n'
if 'fi' in locals():
    report_md += fi.head(10).to_markdown() + '\n'

report_md += '\n## Recommendations\n- Use the model as a screening tool, not a diagnostic tool.\n- Collect more patient data to improve generalization.\n- Consider calibration of predicted probabilities before deployment.\n'

with open(OUT_DIR / 'short_report.md', 'w') as f:
    f.write(report_md)

print('\nShort report saved to', OUT_DIR / 'short_report.md')

# ---------------------------
# End of script
# ---------------------------
print('\nAll outputs saved to', OUT_DIR)
